<a href="https://colab.research.google.com/github/YeweiYuan/Quantum-Pricing/blob/main/iqae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyqpanda

In [7]:
import pyqpanda.pyQPanda as pq
import pyqpanda.utils as pu
from pyqpanda import *
import math
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def QG(qbits,G=pq.H):  #批量对qbits进行gate操作,默认G初始化为哈达玛门//注意qbits类型需要是列表注意这里一开始circuit=pq.create_empty_circuit()传参circuit会报错
  circuit = pq.create_empty_circuit()
  if type(G) == type(pq.H):  
    for i in range(len(qbits)):
      circuit.insert(G(qbits[i]))
  return circuit

def power(circuit,k):  #进行k次circuit操作
  circuitk = pq.create_empty_circuit()
  for j in range(k):
    circuitk.insert(circuit)
  return circuitk

def to_zero(machine,qbits,cbits=0,flag=0):    #把qbits归零（复位），可以用在下面的函数或者一些调试，也可以在一些程序里重复利用qubit,flag为1时为变1 QwQnmd为什么不能把测量写进circuit
  circuit = pq.create_empty_circuit()
  ancprog = pq.create_empty_qprog()
  if cbits == 0:
    cbits = machine.cAlloc_many(len(qbits))
  ancprog.insert(pq.measure_all(qbits,cbits))
  res = machine.run_with_configuration(prog,cbits,1)
  
  for i in range(len(qbits)):
    if int(list(result.keys())[0][i]) == 1-flag:
      circuit.insert(QG([qbits[len(m_qlist)-1-i]],pq.X)) #这里有个坑，观测输出的二进制数是按逆序来的要转回来
  return circuit

In [15]:
'先搞一个算符A，全H门加一个RY门'
def operatorA(qlist):
  l = len(qlist)
  A = pq.create_empty_circuit()
  A.insert(QG(qlist[0:l-1]))
  A.insert(pq.RY(qlist[l-1],np.pi/3))
  return A

In [21]:
l = 2
A = pq.create_empty_circuit()
A.insert(QG(qlist[0:l-1]))
A.insert(pq.RY(qlist[l-1],np.pi/3))
A_matrix = pq.get_matrix(A)
Adagger_matrix = pq.get_matrix(A.dagger())
A_matrix

[(0.6123724356957924+0j),
 (-0.35355339059327245+0j),
 (0.6123724356957924+0j),
 (-0.35355339059327245+0j),
 (0.35355339059327245+0j),
 (0.6123724356957924+0j),
 (0.35355339059327245+0j),
 (0.6123724356957924+0j),
 (0.6123724356957924+0j),
 (-0.35355339059327245+0j),
 (-0.6123724356957924+0j),
 (0.35355339059327245-0j),
 (0.35355339059327245+0j),
 (0.6123724356957924+0j),
 (-0.35355339059327245+0j),
 (-0.6123724356957924+0j)]

In [32]:
A_matrix = np.array(A_matrix).reshape(4,4) 
Adagger_matrix = np.array(Adagger_matrix).reshape(4,4)

In [29]:
A_matrix

array([[ 0.61237244+0.j, -0.35355339+0.j,  0.61237244+0.j,
        -0.35355339+0.j],
       [ 0.35355339+0.j,  0.61237244+0.j,  0.35355339+0.j,
         0.61237244+0.j],
       [ 0.61237244+0.j, -0.35355339+0.j, -0.61237244+0.j,
         0.35355339-0.j],
       [ 0.35355339+0.j,  0.61237244+0.j, -0.35355339+0.j,
        -0.61237244+0.j]])

In [44]:
phi0 = np.array([1/2**(1/2),1/2**(1/2)]) #算符A 1+1维的情况
zero = np.array([1,0])
Sphi0 = np.eye(4) - 2*np.kron(np.outer(phi0,phi0),np.outer(zero,zero))
S0 = np.eye(4) - 2*np.kron(np.outer(zero,zero),np.outer(zero,zero))
Qm = -Sphi0.dot(Adagger_matrix).dot(S0).dot(A_matrix)
Qm.reshape(16,).tolist()

[(-0.7499999999999947-0j),
 (0.433012701892216-0j),
 (0.24999999999999822-0j),
 (0.433012701892216-0j),
 (-0.4330127018922162-0j),
 (-0.7499999999999947-0j),
 (-0.43301270189221625-0j),
 (0.24999999999999814-0j),
 (0.24999999999999822-0j),
 (0.433012701892216-0j),
 (-0.7499999999999947-0j),
 (0.433012701892216-0j),
 (-0.43301270189221625-0j),
 (0.24999999999999814-0j),
 (-0.4330127018922162-0j),
 (-0.7499999999999947-0j)]

In [45]:
def operatorG(qlist):
  Qm = [(-0.7499999999999947-0j),
 (0.433012701892216-0j),
 (0.24999999999999822-0j),
 (0.433012701892216-0j),
 (-0.4330127018922162-0j),
 (-0.7499999999999947-0j),
 (-0.43301270189221625-0j),
 (0.24999999999999814-0j),
 (0.24999999999999822-0j),
 (0.433012701892216-0j),
 (-0.7499999999999947-0j),
 (0.433012701892216-0j),
 (-0.43301270189221625-0j),
 (0.24999999999999814-0j),
 (-0.4330127018922162-0j),
 (-0.7499999999999947-0j)]
  Q = pq.matrix_decompose(qlist,Qm)
  return Q

In [48]:
operatorG(m_qlist)


                                                                                                      ┌───────────────────────────────────────────┐                                               ┌───────────────────────────────────────────┐                                                                                             
q_0:  |0>───────────────────────■────────────────────── ──────────────────────■────────────────────── ┤U4(1.570796,3.141593,2.094395,0.000000).dag├ ──────────────────────■────────────────────── ┤U4(1.570796,0.000000,1.047198,3.141593).dag├ ──────────────────────■────────────────────── ───────────────────────────────────────────── 
          ┌─────────────────────┴─────────────────────┐ ┌─────────────────────┴─────────────────────┐ └─────────────────────┬─────────────────────┘ ┌─────────────────────┴─────────────────────┐ └───────────────────────────────────────────┘ ┌─────────────────────┴─────────────────────┐ ┌───────────────────────────────────────────┐ 


In [138]:
class IQAE:
  def __init__(self, epsilon: float = 2**(-6), # Target precision for estimation target `a`
         alpha: float = 0.1,       # Confidence level, the target probability is 1 - alpha
         method: str = 'cher',     # Statistical method used to estimate the confidence intervals in each iteration
         ratio: float = 2.0,      # Minimal q-ratio for FindNextK  
         operatorA = None, #输入的是需要进行AE的算子A
         qnumber: int = 0,        #A作用的比特数        
         operatorG = None, #grover operater        
         ): 
    '''    
    validate_range('epsilon', epsilon, 0, 0.5)
    validate_range('alpha', alpha, 0, 1)
    validate_in_set('confint_method', confint_method, {'chernoff', 'beta'})
    '''
    self.epsilon = epsilon
    self.alpha = alpha
    self.method = method
    self.ratio = ratio
    self.qnumber = qnumber
    
    self.operatorA = operatorA
    self.operatorG = operatorG

    self.N_max = 32/(1-2*np.sin(np.pi/14))**2 * np.log(2/self.alpha * np.log(np.pi/4/self.epsilon)/np.log(self.ratio))
    self.N_shots = int(self.N_max)  #这里怎么确定N_shots有待考究


  def run(self):
    i_count = 0
    k = [0]
    up = True
    theta_l = 0
    theta_u = np.pi / 2
    rounds_m = int(np.log(np.pi / 8 / self.epsilon)/np.log(self.ratio)) + 1

    res = []

    if self.method == 'cher':
      L_max = (np.arcsin(2/self.N_shots * np.log(2*rounds_m/self.alpha)))**(1/4)
    
    M_add = 0   #k不变时会被累积的出现1的次数
    N_add = 0   #k不变时会被累积的观测总次数

    while theta_u - theta_l > 2 * self.epsilon:
      i_count += 1
      k_next, up = self.FindNextK(k[i_count-1], theta_l, theta_u, up)
      k.append(k_next)
      K = 4*k_next + 2
      if K > int(L_max/self.epsilon)+1:
        N = int(self.N_shots * L_max/self.epsilon/K/10)+1
      else:
        N = self.N_shots
      print(k_next,N)
      M = self.Measure(k_next,N)
      print(M)
      if k[i_count] == k[i_count - 1]:
        M_add += M
        N_add += N
        res = M_add/N_add
      else:
        M_add = 0
        N_add = 0
        res = M/N

      theta_l,theta_u = self.chernoff_confint(res, rounds_m) 

    a_l = np.sin(theta_l)**2
    a_u = np.sin(theta_u)**2

    return a_l,a_u

  def Measure(self,k: int, N: int) -> int: #返回1的次数
    machine = pq.init_quantum_machine(pq.QMachineType.CPU) 
    qlist = machine.qAlloc_many(self.qnumber)
    clist = machine.cAlloc_many(self.qnumber)
    prog = pq.QProg()
    prog.insert(self.operatorA(qlist))
    prog.insert(power(self.operatorG(qlist),k))
    prog.insert(pq.measure_all([m_qlist[self.qnumber-1]],[m_clist[self.qnumber-1]])) #这里只测量最后一个比特
    res = m_machine.run_with_configuration(prog, [m_clist[self.qnumber-1]], N)
    M = res.get('1')
    return M

  def FindNextK(self, k_i: int, theta_l: float, theta_u: float, up: bool) -> (int, bool):
    ratio = self.ratio
    if ratio <= 1:
      raise ValueError('ratio must be larger than 1')  

    K_i = 4 * k_i +2
    theta_min = K_i * theta_l
    theta_max = K_i * theta_u
    K_max = int(np.pi / (theta_u - theta_l))
    K = K_max - (K_max - 2) % 4

    while K >= ratio * K_i:
      q = K / K_i
      if (q*theta_max) % (np.pi*2) <= np.pi and (q*theta_min) % (np.pi*2) <= np.pi:
        K_next = K
        up = True
        k_next = int((K_next-2)/4)
        return k_next, up

      elif (q*theta_max) % (np.pi*2) >= np.pi and (q*theta_min) % (np.pi*2) >= np.pi:
        K_next = K
        up = True
        k_next = int((K_next-2)/4)
        return k_next, up
      K -= 4

    return k_i,up

  def chernoff_confint(self,value: float, max_rounds: int):
    shots = self.N_shots
    alpha = self.alpha
    eps = np.sqrt(3 * np.log(2 * max_rounds / alpha) / shots)
    lower = np.maximum(0, value - eps)
    upper = np.minimum(1, value + eps)
    return lower, upper


In [139]:
test= IQAE(0.1,0.1,'cher',2.0,operatorA,2,operatorG)

In [140]:
test.run()

0 424
114
1 26
26


(0.5529457032749244, 0.7080734182735712)

In [ ]:
32/(1-2*np.sin(np.pi/14))**2 * np.log(2/(0.1) * np.log(np.pi/4/(1/2**5))/np.log(2))

782.2522956341725

In [ ]:
m_machine = pq.init_quantum_machine(pq.QMachineType.CPU) 
m_qlist = m_machine.qAlloc_many(3)
m_clist = m_machine.cAlloc_many(3)
cir = pq.create_empty_circuit()
cir.insert(pq.H(m_qlist[0]))


          ┌─┐ 
q_0:  |0>─┤H├ 
          └─┘ 

In [ ]:
cir.begin()

In [4]:
m_machine = pq.init_quantum_machine(pq.QMachineType.CPU) 
m_qlist = m_machine.qAlloc_many(2)
m_clist = m_machine.cAlloc_many(2)
cir = pq.create_empty_circuit()
cir.insert(QG(m_qlist,pq.X))
cir.insert(pq.Z(m_qlist[1]).control(m_qlist[0]))
cir.insert(QG(m_qlist,pq.X))
ma = pq.get_matrix(cir)

In [5]:
qlist = m_machine.qAlloc_many(2)
out_cir = pq.matrix_decompose(qlist,ma)
out_cir


                                                                                                      ┌───────────────────────────────────────────┐ ┌───────────────────────────────────────────┐ 
q_2:  |0>───────────────────────■────────────────────── ──────────────────────■────────────────────── ┤U4(4.712389,3.141593,0.000000,0.000000).dag├ ┤U4(4.712389,3.141593,0.000000,0.000000).dag├ 
          ┌─────────────────────┴─────────────────────┐ ┌─────────────────────┴─────────────────────┐ └─────────────────────┬─────────────────────┘ └───────────────────────────────────────────┘ 
q_3:  |0>─┤U4(1.570796,3.141593,0.000000,0.000000).dag├ ┤U4(1.570796,3.141593,0.000000,0.000000).dag├ ──────────────────────■────────────────────── ───────────────────────────────────────────── 
          └───────────────────────────────────────────┘ └───────────────────────────────────────────┘                                                                                             

In [10]:
prog = pq.QProg()
prog.insert(out_cir)
res = m_machine.prob_run_tuple_list(prog,qlist,-1)

In [11]:
res

[(0, 1.0), (1, 0.0), (2, 0.0), (3, 0.0)]

In [12]:
pq.get_matrix(m_qlist[1])

TypeError: ignored

In [14]:
np.array([1/2**(1/2),1/2**(1/2)])

array([0.70710678, 0.70710678])